In [50]:
import pandas as pd
import numpy as np
import re
import urllib.parse

In [51]:
df=pd.read_csv('new_data.csv')

/var/folders/9g/mm9pdxg52dn6rg0rtfk14syc0000gn/T/ipykernel_38772/889337693.py:1: DtypeWarning: Columns (102,116,122,123,124,125,126,128,129) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('new_data.csv')


In [52]:
df.columns

Index(['Unnamed: 0', 'Available as of', 'Property name', 'Construction year',
       'Floor', 'Number of floors', 'Building condition',
       'Number of frontages', 'Covered parking spaces',
       'Outdoor parking spaces',
       ...
       'Percentage rented', 'Age of annuitant', 'EPC description',
       'Venue of the sale', 'Single session', 'Terms of visit',
       'Starting price', '0', 'Maximum duration of annuity',
       'Date of the sale'],
      dtype='object', length=130)

In [53]:
df.shape

(19291, 130)

In [54]:
filtered_columns=["Unnamed: 0", "Zip-code", "City", "Type of Property", 
                  "Subtype of Property", "Price", "Construction year", "Bedrooms","Living area",
                  "Kitchen type", "Furnished", "How many fireplaces?", "Terrace","Terrace surface",
                  "Garden","Garden surface","Surface of the plot","Number of frontages",
                  "Swimming pool", "Building condition", "Elevator", "Basement", "Cadastral income",
                  "Primary energy consumption","CO₂ emission", "Energy class", "Heating type"]
    
df = df[filtered_columns]

In [55]:
df=df.drop_duplicates()

In [56]:
df=df.rename(columns = {'Unnamed: 0':'Property ID', 'Zip-code':'Zip code'})

In [57]:
df=df.rename(columns = {'Price':'Price (€)', 'Living area':'Living area (m²)', 'Terrace surface':'Terrace surface (m²)', 
                        'Garden surface':'Garden surface (m²)', 'Surface of the plot': 'Surface of the plot (m²)', 'Cadastral income':'Cadastral income (€)',
                        'Primary energy consumption':'Primary energy consumption (kWh/m²)', 'CO₂ emission':'CO₂ emission (kg CO₂/m²)',
                          })

In [58]:
df=df.drop_duplicates()

In [59]:
df['How many fireplaces?']=df['How many fireplaces?'].replace(np.nan,0,regex=True)
df['How many fireplaces?']=df['How many fireplaces?'].replace('Yes',1,regex=True)
df['How many fireplaces?']=df['How many fireplaces?'].replace('No',0,regex=True)
df['Swimming pool']=df['Swimming pool'].replace(np.nan,0,regex=True)
df['Swimming pool']=df['Swimming pool'].replace('Yes',1,regex=True)
df['Swimming pool']=df['Swimming pool'].replace('No',0,regex=True)
df['Elevator']=df['Elevator'].replace(np.nan,0,regex=True)
df['Elevator']=df['Elevator'].replace('Yes',1,regex=True)
df['Elevator']=df['Elevator'].replace('No',0,regex=True)
df['Basement']=df['Basement'].replace(np.nan,0,regex=True)
df['Basement']=df['Basement'].replace('Yes',1,regex=True)
df['Basement']=df['Basement'].replace('No',0,regex=True)
df['Furnished']=df['Furnished'].replace(np.nan,0,regex=True)
df['Furnished']=df['Furnished'].replace('Yes',1,regex=True)
df['Furnished']=df['Furnished'].replace('No',0,regex=True)


In [60]:
df=df.dropna(subset=['Price (€)', 'Bedrooms', 'Living area (m²)', 'Subtype of Property'])


In [61]:
df['Price (€)'] = df['Price (€)'].str.split(' ').str[-2]
df['Price (€)'] = pd.to_numeric(df['Price (€)'])
df['Living area (m²)'] = df['Living area (m²)'].str.split(' ').str[0]
df['Garden surface (m²)'] = df['Garden surface (m²)'].str.split(' ').str[0]
df['Terrace surface (m²)'] = df['Terrace surface (m²)'].str.split(' ').str[0]
df['Surface of the plot (m²)'] = df['Surface of the plot (m²)'].str.split(' ').str[0]
df['Cadastral income (€)'] = df['Cadastral income (€)'].str.split(' ').str[-2]
df['Primary energy consumption (kWh/m²)'] = df['Primary energy consumption (kWh/m²)'].str.split(' ').str[0]
df['CO₂ emission (kg CO₂/m²)'] = df['CO₂ emission (kg CO₂/m²)'].str.split(' ').str[0]

In [62]:
filter_G = df["Garden surface (m²)"].isnull()
df.loc[~filter_G,'Garden'] = 'Yes'
df.loc[df['Garden'] == 'Yes', 'Garden'] = 1
df.loc[df["Garden"].isnull(), 'Garden'] = 0
df.loc[df["Garden surface (m²)"].isnull(), 'Garden surface (m²)'] = 0

In [63]:
filter_G = df["Terrace surface (m²)"].isnull()
df.loc[~filter_G,'Terrace'] = 'Yes'
df.loc[df['Terrace'] == 'Yes', 'Terrace'] = 1
df.loc[df["Terrace"].isnull(), 'Terrace'] = 0
df.loc[df["Terrace surface (m²)"].isnull(), 'Terrace surface (m²)'] = 0

In [64]:
df['Primary energy consumption (kWh/m²)']=df['Primary energy consumption (kWh/m²)'].replace('Not',np.nan)
df['CO₂ emission (kg CO₂/m²)']=df['CO₂ emission (kg CO₂/m²)'].replace('Not',np.nan)
df['Energy class']=df['Energy class'].replace('Not',np.nan)

In [65]:
df = df.astype({"Living area (m²)":"float", "Terrace":"float", "Garden":"float",
                "Terrace surface (m²)":"float",
                "Garden surface (m²)":"float","Surface of the plot (m²)":"float",
                "Primary energy consumption (kWh/m²)":"float", "Cadastral income (€)":"float",
                "CO₂ emission (kg CO₂/m²)":"float"})

In [66]:
df['Energy class']=df['Energy class'].replace('Not specified',np.nan)

In [67]:
kitchen_type_scale={'USA hyper equipped':3, 'USA installed':2, 'USA semi equipped':1, 'USA uninstalled':0,
 'Hyper equipped':3, 'Installed':2, 'Semi equipped':1, 'Not installed':0}

df['Kitchen type scale'] = df['Kitchen type'].map(kitchen_type_scale)

In [68]:
building_condition_scale={'As new':6,'Just renovated':5, 'Good':4, 'To renovate':2,
 'To restore':1, 'To be done up':3}

df['Building condition scale'] = df['Building condition'].map(building_condition_scale)

In [69]:
def clean_city_name(city):
    # Remove quotes
    city = city.replace('"', '')
    # Decode URL encoding
    city = urllib.parse.unquote(city)
    return city

df['City'] = df['City'].apply(clean_city_name)

In [70]:
df = df.applymap(lambda x: x.strip() if isinstance(x, str) else x)

In [71]:
df.to_csv("updated_cleaned_data.csv", index=False)

In [72]:
df.head(30)

,Property ID,Zip code,City,Type of Property,Subtype of Property,Price (€),Construction year,Bedrooms,Living area (m²),Kitchen type,...,Building condition,Elevator,Basement,Cadastral income (€),Primary energy consumption (kWh/m²),CO₂ emission (kg CO₂/m²),Energy class,Heating type,Kitchen type scale,Building condition scale
0,10583186,1341,céroux-mousty,apartment,duplex,415000,2016.0,2.0,139.0,USA hyper equipped,...,As new,1,0,NaN,116.0,NaN,B,Gas,3.0,6.0
1,10463775,1180,uccle,apartment,penthouse,1650000,2021.0,3.0,235.0,USA hyper equipped,...,As new,1,1,NaN,62.0,9.0,B,Gas,3.0,6.0
2,10582739,1180,uccle,apartment,flat studio,159000,1977.0,1.0,30.0,Installed,...,Good,1,0,666.0,316.0,NaN,F,Electric,2.0,4.0
3,10582752,4300,waremme,house,mansion,495000,1918.0,6.0,250.0,Installed,...,As new,0,0,1105.0,130.0,25.0,B,Gas,2.0,6.0
4,10579605,1332,rixensart genval,house,villa,530000,1928.0,4.0,176.0,USA hyper equipped,...,Good,0,1,880.0,394.0,69.0,E,Gas,3.0,4.0
5,10578260,1000,bruxelles,apartment,penthouse,340000,1972.0,2.0,70.0,USA hyper equipped,...,Just renovated,1,1,1175.0,647.0,142.0,G,Gas,3.0,5.0
6,10577975,9000,gent,apartment,penthouse,459000,1978.0,2.0,95.0,Installed,...,As new,1,0,1316.0,131.0,NaN,B,Gas,2.0,6.0
7,10577525,1340,ottignies-louvain-la-neuve,house,villa,650000,1989.0,4.0,248.0,USA hyper equipped,...,Good,1,1,3550.0,220.0,55.0,C,Fuel oil,3.0,4.0
8,10576930,1020,laeken,house,apartment block,730000,1952.0,6.0,300.0,Hyper equipped,...,To be done up,1,0,2992.0,644.0,129.0,G,Gas,3.0,3.0
9,10581309,6250,aiseau,house,villa,540000,1971.0,5.0,285.0,Installed,...,Good,0,0,1955.0,305.0,NaN,D,Gas,2.0,4.0
